# PHI augmentation : i2b2-2014 dataset processing

The i2b2 2014 de-identification dataset can be accessed from https://portal.dbmi.hms.harvard.edu.

The data processing mainly refers to the guidance from this link:
https://github.com/juand-r/entity-recognition-datasets/tree/master/data/i2b2_2014

1.Converting training-PHI-Gold-Set1.tar, training-PHI-Gold-Set2.tar, testing-PHI-Gold-fixed.tar into BRAT format by:

In [ ]:
python xml_to_brat.py

2.Combine all the files, then randomly split the files into train, validation, test set with the proportion of 7:1:2.

3.Using Neuroner package (https://github.com/Franck-Dernoncourt/NeuroNER) to get train, validation, test file in BIO format.

4.As the processed file exists a few errors, we fix it.

In [3]:
"""
123  B-STREET

Alaska   B-STREET
Street  I-STREET

############### convert the above case into normal one #################

123  B-STREET
Alaska   I-STREET
Street  I-STREET
"""

file_path = 'C:/Users/zhou/Desktop/valid_spacy.txt'
file_preserve = []
write_file = 'C:/Users/zhou/Desktop/valid_spacy_new.txt'

with open(file_path, 'r', encoding='utf-8') as f_r:
    content = f_r.readlines()
    for j in range(0, len(content)):
        line = content[j]
        file_preserve.append(line)
        if j >= 2 and len(content[j-1].split()) == 0:   # previous line is empty
            previsou_label = content[j - 2].strip().split()[-1]
            previsou_end_position = content[j - 2].strip().split()[-2]
            label = content[j].strip().split()[-1]
            start_position = content[j].strip().split()[-3]  # be cautious -3
            if previsou_label == 'O' or label == 'O':
                continue
            else:
                previsou_label_type = previsou_label.split('-')[1]
                label_type = label.split('-')[1]

            if previsou_label_type == label_type and label_type != 'DATE' and int(start_position) <= (int(previsou_end_position)+3):
                file_preserve.pop()  # delete this line
                file_preserve.pop()  # delete empty line
                newly_adjusted_label = 'I-' + previsou_label_type
                new_line = content[j].replace(label, newly_adjusted_label)
                file_preserve.append(new_line)

with open(write_file, 'w', encoding='utf-8') as f_ww:
    for line in file_preserve:
        f_ww.writelines(line)

If want to conduct PHI augmentation, run the step 5,6,7; If not, skip to step 7.

5.Replace the label of low frequency PHI type, e.g., FAX, with 'O'. 
And merge four types of location labels into 'LOCATION'.

In [2]:
other_PHI = ['FAX', 'EMAIL', 'URL', 'DEVICE', 'HEALTHPLAN', 'BIOID', 'LOCATION_OTHER']
location_phi = ['CITY', 'COUNTRY', 'STATE', 'STREET']

reserve_content = []
file_path = 'C:/Users/zhou/Desktop/train_spacy_new.txt'
file_path_write = 'C:/Users/zhou/Desktop/processed_2014_train.txt'

with open(file_path, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for line in content:
        if len(line.split()) == 0:  # preserve empty line
            reserve_content.append(line)
        else:
            words = line.split()
            label = words[-1]
            if label == 'O':
                reserve_content.append(line)
            else:
                label_type = label.split('-')[1]
                if label_type in other_PHI:
                    line = line.replace(label, 'O')  # replace the whole label
                    reserve_content.append(line)
                elif label_type in location_phi:
                    line = line.replace(label_type, 'LOCATION')  # only replace the label_type
                    reserve_content.append(line)
                else:
                    reserve_content.append(line)

with open(file_path_write, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.write(line)

6.Conduct PHI augmentation, run the folowing code.
Notice that we replace PHI entity in fine-grainity (in PHI type level).

In [3]:
import random

file_path_entity = 'C:/Users/zhou/Desktop/all_PHI_from_internet/'
labels_in_2006 = ['PATIENT', 'DOCTOR', 'HOSPITAL', 'IDNUM', 'DATE', 'PHONE', 'AGE', 'ZIP',
                  'MEDICALRECORD', 'ORGANIZATION', 'PROFESSION', 'USERNAME', 'LOCATION']

label_content = [[] for i in range(len(labels_in_2006))]
for k in range(len(labels_in_2006)):
    file_path = file_path_entity + labels_in_2006[k] + '.txt'
    with open(file_path, 'r', encoding='utf-8') as f_w:
        content = f_w.readlines()
        for j in content:
            label_content[k].append(j.strip())


def replace_PHI(label, line_number):
    preserve_list = []
    if label in labels_in_2006:
        label_index = labels_in_2006.index(label)
        words_list = label_content[label_index]

        random.seed(global_seed)  # every epoch, it produces different dataset, also reimplementable.
        random.shuffle(words_list)
        words_num = len(words_list)

        random.seed(line_number)  # line_number is defined for specific PHI instance
        random_index = random.randint(0, words_num - 1)
        random_w = words_list[random_index].strip()  # words_list is different at different epoch

        words_num = random_w.split()
        if len(words_num) == 1:
            new_line = random_w+'   '+'B-'+label
            preserve_list.append(new_line)
        if len(words_num) > 1:
            for i in range(len(words_num)):
                if i == 0:
                    new_line = words_num[i]+'   '+'B-'+label
                    preserve_list.append(new_line)
                if i > 0:
                    new_line = words_num[i]+'   '+'I-'+label
                    preserve_list.append(new_line)
    return preserve_list


file_path_2014 = 'C:/Users/zhou/Desktop/processed_2014_train.txt'
global_seed = 2
repalced_file_path = 'C:/Users/zhou/Desktop/new_replaced_2014_train_seed'+str(global_seed)+'.txt'


reserve_content = []
with open(file_path_2014, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for row in range(0, len(content)-1):
        if len(content[row]) == 1:  # cope with empty line
            reserve_content.append('')
        if len(content[row]) != 1 and len(content[row+1]) != 1:
            all_token_Next = content[row + 1].split()
            all_token = content[row].split()
            this_line_label = all_token[-1]
            next_line_label = all_token_Next[-1]
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue
            if this_line_label.startswith('B-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('B-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('I-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('I-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                # print(label)
                new_words = replace_PHI(label, row)
                # print(new_words)
                for k in new_words:
                    reserve_content.append(k)

        if len(content[row]) != 1 and len(content[row+1]) == 1:
            all_token = content[row].split()
            this_line_label = all_token[-1]
            if this_line_label.startswith('B-') or this_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue


with open(repalced_file_path, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.writelines(line+'\n')

7.Combine PHI into five PHI categories.

In [6]:
label_contact = 'PHONE'   # delete FAX, EMAIL,URL
label_date = 'DATE'
label_id = ['IDNUM', 'MEDICALRECORD']  # delete DEVICE, HEALTHPLAN, BIOID
label_location = ['HOSPITAL', 'CITY', 'STREET', 'ZIP', 'STATE', 'COUNTRY', 'ORGANIZATION', 'LOCATION']  
label_name = ['PATIENT', 'DOCTOR', 'USERNAME']
other_PHI = ['AGE','FAX', 'EMAIL', 'URL', 'DEVICE', 'HEALTHPLAN', 'BIOID', 'LOCATION_OTHER', 'PROFESSION']

reserve_content = []
file_path = 'C:/Users/zhou/Desktop/valid_spacy_new.txt'
file_path_write = 'C:/Users/zhou/Desktop/5_category_2014_valid.txt'

with open(file_path, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for line in content:
        if len(line.split()) == 0 :  # preserve empty line
            reserve_content.append(line)
        else:
            words = line.split()
            label = words[-1]
            if label == 'O':
                reserve_content.append(line)
            else:
                label_type = label.split('-')[1]
                if label_type in label_location:
                    new_label = label.replace(label_type, 'LOCATION')  # B-HOSPITAL --> B-LOCATION
                    new_line = line.replace(label, new_label)  # XXX B-LOCATION
                    reserve_content.append(new_line)
                elif label_type == label_contact:
                    new_label = label.replace(label_type,'CONTACT')
                    new_line = line.replace(label, new_label)
                    reserve_content.append(new_line)
                elif label_type == label_date:
                    reserve_content.append(line)
                elif label_type in label_name:
                    new_label = label.replace(label_type, 'NAME')
                    new_line = line.replace(label, new_label)
                    reserve_content.append(new_line)
                elif label_type in label_id:
                    line = line.replace(label_type, 'ID')
                    reserve_content.append(line)
                else:
                    assert label_type in other_PHI
                    line = line.replace(label, 'O')  # replace the whole label
                    reserve_content.append(line)

with open(file_path_write, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.write(line)

8.If want to conduct context augmentation, turn to relevant step.